In [1]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objs as go
import plotly.express as px

# Step 1: Load and preprocess the dataset
df = pd.read_csv('../excels/2017-2018-clean.csv', parse_dates=['Date/Time'])

# Drop rows with NaN values
df = df.dropna()

# Rename columns for Prophet
df.rename(columns={
    'Date/Time': 'ds',
    '100m_N Avg [m/s]': 'y',
    '100m_N Std [m/s]': '100m_N_Std',
    'Pressure [mbar]': 'Pressure',
    '98m WV [°]': 'WV_98m',
    '78m WV [°]': 'WV_78m',
    '48m WV [°]': 'WV_48m',
    'Temp 5m [°C]': 'Temp_5m',
    'Hum 5m [%]': 'Hum_5m'
}, inplace=True)

# Step 2: Prepare the data for Prophet
prophet_model = Prophet()
prophet_model.add_regressor('100m_N_Std')
prophet_model.add_regressor('Pressure')
prophet_model.add_regressor('WV_98m')
prophet_model.add_regressor('WV_78m')
prophet_model.add_regressor('WV_48m')
prophet_model.add_regressor('Temp_5m')
prophet_model.add_regressor('Hum_5m')

# Step 3: Fit the Prophet model
prophet_model.fit(df)

# Step 4: Make future predictions
# Create a dataframe with future dates for the next 30 days
future = prophet_model.make_future_dataframe(periods=6*24*30, freq='10min')  # predicting for the next 30 days

# Add the same regressors to the future dataframe
last_row = df.iloc[-1]
future['100m_N_Std'] = last_row['100m_N_Std']
future['Pressure'] = last_row['Pressure']
future['WV_98m'] = last_row['WV_98m']
future['WV_78m'] = last_row['WV_78m']
future['WV_48m'] = last_row['WV_48m']
future['Temp_5m'] = last_row['Temp_5m']
future['Hum_5m'] = last_row['Hum_5m']

# Make predictions
forecast = prophet_model.predict(future)

# Create interactive plot with Plotly
fig = go.Figure()

# Add actual data trace
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='markers', name='Actual', marker=dict(color='black')))

# Add forecasted data trace
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast', line=dict(color='blue')))

# Add confidence interval traces
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', name='Upper Confidence Interval',
                         line=dict(color='lightblue'), fill=None))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', name='Lower Confidence Interval',
                         line=dict(color='lightblue'), fill='tonexty', fillcolor='rgba(173, 216, 230, 0.2)'))

# Update layout for interactivity
fig.update_layout(
    title='Wind Speed Forecast',
    xaxis_title='Date',
    yaxis_title='Wind Speed (m/s)',
    xaxis_rangeslider_visible=True,
    xaxis_rangeselector=dict(
        buttons=list([
            dict(count=1, label='1m', step='month', stepmode='backward'),
            dict(count=6, label='6m', step='month', stepmode='backward'),
            dict(count=1, label='YTD', step='year', stepmode='todate'),
            dict(count=1, label='1y', step='year', stepmode='backward'),
            dict(step='all')
        ])
    ),
    legend=dict(x=0.01, y=0.99),
    hovermode='x'
)

# Show the plot
fig.show()


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
20:45:24 - cmdstanpy - INFO - Chain [1] start processing
20:48:05 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed